In the section [Preparation of speech and text data](https://github.com/pytorch/fairseq/tree/master/examples/wav2vec/unsupervised#preparation-of-speech-and-text-data) of the readme, it says:

> Similar to [wav2vec 2.0](https://github.com/pytorch/fairseq/blob/master/examples/wav2vec/README.md),  data folders contain {train,valid,test}.{tsv,wrd,phn} files, where audio paths are stored in tsv files, and word, letter or phoneme transcriptions are stored in .{wrd,ltr,phn}.

...where are these files explained? This is my best guess:

In [1]:
%cd /tmp

/tmp


In [2]:
!git clone https://github.com/pytorch/fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 28101, done.
remote: Counting objects: 100% (522/522), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 28101 (delta 254), reused 398 (delta 197), pack-reused 27579
Receiving objects: 100% (28101/28101), 11.85 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (21069/21069), done.


In [3]:
%%capture
!pip install phonemizer

In [4]:
%%capture
!apt-get -y install espeak

In [5]:
import os
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [6]:
%%capture
!apt-get -y install zsh

This is just my best guess at what the `.wrd` files contain

In [7]:
!cat /kaggle/input/download-common-voice-swedish/cv-corpus-6.1-2020-12-11/sv-SE/test.tsv | awk -F'\t' '{print $3}'|grep -v '^sentence$' | perl -C7 -ane 'chomp;$_=lc($_);s/[^\p{L}\p{N}\p{M}'"\'"' \-]/ /g;s/  +/ /g;s/ $//;s/^ //;print "$_\n";' > /kaggle/working/test.wrd
!cat /kaggle/input/download-common-voice-swedish/cv-corpus-6.1-2020-12-11/sv-SE/dev.tsv | awk -F'\t' '{print $3}'|grep -v '^sentence$' | perl -C7 -ane 'chomp;$_=lc($_);s/[^\p{L}\p{N}\p{M}'"\'"' \-]/ /g;s/  +/ /g;s/ $//;s/^ //;print "$_\n";' > /kaggle/working/valid.wrd
!cat /kaggle/input/download-common-voice-swedish/cv-corpus-6.1-2020-12-11/sv-SE/train.tsv | awk -F'\t' '{print $3}'|grep -v '^sentence$' | perl -C7 -ane 'chomp;$_=lc($_);s/[^\p{L}\p{N}\p{M}'"\'"' \-]/ /g;s/  +/ /g;s/ $//;s/^ //;print "$_\n";' > /kaggle/working/train.wrd


In [8]:
%cd fairseq/examples/wav2vec/unsupervised/scripts

/tmp/fairseq/examples/wav2vec/unsupervised/scripts


This is probably completely wrong

In [9]:
!for i in train test valid; do cat /kaggle/working/$i.wrd | python wrd_to_ltr.py > /kaggle/working/$i.ltr;done

In [10]:
!head /kaggle/working/train.ltr

v a d | ä r | d e t | i | e u r o |
d u | s k a | v e t a | a t t | d e t | ä r | d u | s o m | h a r | f e l |
g å | n e r | p å | k n ä |
f ö r s t | m å s t e | j a g | s l å | s ö n d e r | d e n | d ä r | s t o r a | s k r o t h ö g e n |
d e t | b l i r | s v å r t |
v a d | f ö r | j ä v l a | f r å g a | ä r | d e t |
j a g | å t e r v ä n d e r | i n t e | t i l l | s k i t h å l e t |
t i t t a | p å | s ö m m a r n a |
f e s | d u | p r e c i s |
a k t r i s e r | h a r | e t t | b ä s t | f ö r e d a t u m |


In [11]:
# | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -o $target_dir/phones.txt -p ' ' -w '' -l $ph_lg -j 70 --language-switch remove-flags
!for i in train test valid; do cat /kaggle/working/$i.wrd | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -o /kaggle/working/$i.phn -p ' ' -w '' -l sv  -j 70 --language-switch remove-flags 2> /kaggle/working/$i.phn.err;done

In [12]:
!for i in /kaggle/working/*.err;do echo $i; cat $i;done

/kaggle/working/test.phn.err
[WARNING] 1 utterances containing language switches on lines 81
[WARNING] extra phones may appear in the "sv" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
/kaggle/working/train.phn.err
[WARNING] 2 utterances containing language switches on lines 254, 1457
[WARNING] extra phones may appear in the "sv" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)
/kaggle/working/valid.phn.err
[WARNING] 1 utterances containing language switches on lines 1831
[WARNING] extra phones may appear in the "sv" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)


In [13]:
!cat /kaggle/working/test.wrd|awk 'BEGIN{ln=1}{if(ln==81){print $0};ln++}'
!cat /kaggle/working/train.wrd|awk 'BEGIN{ln=1}{if(ln==254||ln==1457){print $0};ln++}'
!cat /kaggle/working/valid.wrd|awk 'BEGIN{ln=1}{if(ln==1831){print $0};ln++}'

det är taskigt
och så unik design
internet slutade fungera
det finns inget internet


In [14]:
!cat /kaggle/working/test.phn|awk 'BEGIN{ln=1}{if(ln==81){print $0};ln++}'
!cat /kaggle/working/train.phn|awk 'BEGIN{ln=1}{if(ln==254||ln==1457){print $0};ln++}'
!cat /kaggle/working/valid.phn|awk 'BEGIN{ln=1}{if(ln==1831){print $0};ln++}'

d eː t ɛː r  t a s k ɪ ɡ t  
ɔ k s oː ɵ n iː k  d ɪ z aɪ n  
 ɪ n t ə n ɛ t  s l ʉ t a d ə f ɵ n ɡ eː r a 
d eː t f ɪ n s ɪ ŋ ə t  ɪ n t ə n ɛ t  


"design" and "internet" are clearly the English words that are causing the switch in their respective sentences, but I'm not sure what the problem in test.wrd is: "taskigt"?
* [design](https://en.wiktionary.org/wiki/design#Swedish) `/dɛˈsajn/`
* [internet](https://en.wiktionary.org/wiki/internet#Swedish) `/ˈɪntɛrnɛt/, /ɪntɛrˈnɛt/`